In [2]:
import functools        # 为函数式编程设计
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 1us/step


In [4]:
np.set_printoptions(precision=3, suppress=True)     # 设置输出样式

In [25]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [9]:
LABEL_COLUMNS = "survived"
LABELS = [0, 1]

In [10]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name=LABEL_COLUMNS,
        na_value="?",
        num_epochs=1,
        ignore_errors=True
    )
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

dataset中的每个条目都是一个批次，用一个元组（多样本，多标签）表示。样本的数据组织形式是以 __列__ 为主的张量。

In [11]:
examples, labels = next(iter(raw_train_data))
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'female', b'female', b'male', b'male', b'male', b'male',
       b'female', b'female', b'female', b'female', b'male', b'male'],
      dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([23., 18., 18., 28., 38.,  3., 51., 28., 22., 11., 46., 20.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([3, 0, 1, 0, 0, 1, 1, 0, 0, 4, 0, 0])>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([2, 2, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0])>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([263.   ,  13.   , 108.9  ,   8.05 , 153.462,  26.   ,  77.958,
         7.733,   7.775,  31.275,  79.2  ,   9.225], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'First', b'Second', b'First', b'Third', b'First', b'Second',
       b'First', b'Third', b'Third', b'Third', b'First', b'Third'],
      dtype

## 数据预处理

### 分类数据
分类数据的列只能在有限的集合中取值

In [13]:
CATEGORIES = {
    "sex": ["male", "female"],
    "class": ["First", "Second", "Third"],
    "deck": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"],
    "embark_town": ["Cherbourg", "Southhampton", "Queenstown"],
    "alone": ["y", "n"]
}

In [14]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab
    )
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [15]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

### 连续数据
连续数据需要标准化

In [16]:
def process_continuous_data(mean, data):
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

In [17]:
MEANS = {
    "age": 29.631308,
    "n_siblings_spouses": 0.545455,
    "parch": 0.379585,
    "fare": 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [18]:
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000027B4BC7AEE8>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000027B4BC7AEE8>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000027B4BC7AEE8>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000027B4BC7AEE8>, 34.385399))]

## 创建预处理层
两个特征列的集合相加，并传给API，创建一个进行预处理的输入层

In [19]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numerical_columns)

## 构建模型

In [20]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

## 训练、评估和预测

In [21]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [22]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================] - 1s 21ms/step - loss: 0.5373 - accuracy: 0.7496
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4293 - accuracy: 0.8022
Epoch 3/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4150 - accuracy: 0.8166
Epoch 4/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4057 - accuracy: 0.8214
Epoch 5/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3962 - accuracy: 0.8246
Epoch 6/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3895 - accuracy: 0.8230
Epoch 7/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3830 - accuracy: 0.8357
Epoch 8/20
53/53 [==============================] - 0s 2ms/step -

In [23]:
test_loss, test_accuracy = model.evaluate(test_data)
print("\n\nTest Loss {}, Test Accuracy {}".format(test_loss, test_accuracy))

22/Unknown - 0s 14ms/step - loss: 0.4543 - accuracy: 0.8144

Test Loss 0.45434276962822134, Test Accuracy 0.814393937587738


In [24]:
# 使用tf.keras.Model.predict推断一个或多个批次的标签
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.2%}".format(prediction[0]),
    " | Actual outcome: ",
    ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 90.02%  | Actual outcome:  DIED
Predicted survival: 25.21%  | Actual outcome:  DIED
Predicted survival: 4.94%  | Actual outcome:  DIED
Predicted survival: 85.54%  | Actual outcome:  DIED
Predicted survival: 83.89%  | Actual outcome:  DIED
Predicted survival: 8.12%  | Actual outcome:  DIED
Predicted survival: 6.39%  | Actual outcome:  DIED
Predicted survival: 18.59%  | Actual outcome:  DIED
Predicted survival: 68.48%  | Actual outcome:  SURVIVED
Predicted survival: 88.61%  | Actual outcome:  DIED
